# Sample code for evaluation of anipose calibrations

In [1]:
from b06_source.camera_calibration import CalibrationForAnipose3DTracking, SingleCamDataForAnipose, TestPositionsGroundTruth

from pathlib import Path
import matplotlib.pyplot as plt
from b06_source.load_config import read_config

# for creating an interactive 3D plot
#%matplotlib widget

In [2]:
dummy_filepath = Path('random.mp4')
charuco_folder = Path('./test_data/Charuco_220825/')

intrinsic_calibrations_dir = Path('test_data/intrinsic_calibrations/')

In [3]:
config = read_config("config.yaml")

### To switch between different calibrations, use one of the following paths:
- `'test_data/evaluate_calibration/calibration_00/'`
- `'test_data/evaluate_calibration/calibration_01/'`
- `'test_data/evaluate_calibration/calibration_02/'`

In [4]:
test_data_dir = Path('./test_data/Charuco_220825/')



## Create individual camera objects with intrinsic calibrations & 2D coordinates of detected markers

In [5]:
top_cam = SingleCamDataForAnipose(cam_id = 'Top', filepath_synchronized_calibration_video = charuco_folder.joinpath('Top_cam_synchronized_for_calibration_all_parts.mp4'))
top_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Top_manual_test_position_marker_fake.h5'))
top_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Top_checkerboard_intrinsic_calibration_results.p'))

bottom_cam = SingleCamDataForAnipose(cam_id = 'Bottom', filepath_synchronized_calibration_video = charuco_folder.joinpath('Bottom_cam_synchronized_for_calibration_all_parts.mp4'))
bottom_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Bottom_manual_test_position_marker_fake.h5'))
bottom_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Bottom_checkerboard_intrinsic_calibration_results.p'))

side1_cam = SingleCamDataForAnipose(cam_id = 'Side1', filepath_synchronized_calibration_video = charuco_folder.joinpath('Side1_cam_synchronized_for_calibration_all_parts.mp4'))
side1_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Side1_manual_test_position_marker_fake.h5'))
side1_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Side1_checkerboard_intrinsic_calibration_results.p'))

side2_cam = SingleCamDataForAnipose(cam_id = 'Side2', filepath_synchronized_calibration_video = charuco_folder.joinpath('Side2_cam_synchronized_for_calibration_all_parts.mp4'))
side2_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Side2_manual_test_position_marker_fake.h5'))
side2_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Side2_checkerboard_intrinsic_calibration_results.p'))

ground1_cam = SingleCamDataForAnipose(cam_id = 'Ground1', filepath_synchronized_calibration_video = charuco_folder.joinpath('Ground1_cam_synchronized_for_calibration_all_parts.mp4'))
ground1_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Ground1_manual_test_position_marker_fake.h5'))
ground1_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Ground1_checkerboard_intrinsic_calibration_results.p'))

ground2_cam = SingleCamDataForAnipose(cam_id = 'Ground2', filepath_synchronized_calibration_video = charuco_folder.joinpath('Ground2_cam_synchronized_for_calibration_all_parts.mp4'))
ground2_cam.load_test_position_markers_df_from_dlc_prediction(filepath_deeplabcut_prediction = test_data_dir.joinpath('Ground2_manual_test_position_marker_fake.h5'))
ground2_cam.load_intrinsic_camera_calibration(filepath_intrinsic_calibration = intrinsic_calibrations_dir.joinpath('Ground2_checkerboard_intrinsic_calibration_results.p'))

single_cams = [bottom_cam, ground1_cam, ground2_cam, side1_cam, side2_cam, top_cam]
#single_cams = [ground1_cam, ground2_cam, side1_cam, side2_cam, top_cam]

User info: since no other information were provided, "cropping_offsets" were set to the corresponding default values: (0, 0).
User info: since no other information were provided, "flipped_horizontally" & "flipped_vertically" were set to the corresponding default values: False.
User info: since no other information were provided, "degrees_rotated_clockwise" were set to the corresponding default values: 0.
User info: since no other information were provided, "cropping_offsets" were set to the corresponding default values: (0, 0).
User info: since no other information were provided, "flipped_horizontally" & "flipped_vertically" were set to the corresponding default values: False.
User info: since no other information were provided, "degrees_rotated_clockwise" were set to the corresponding default values: 0.
User info: since no other information were provided, "cropping_offsets" were set to the corresponding default values: (0, 0).
User info: since no other information were provided, "flip

## Load calibration, triangulate markers & plot

In [6]:
anipose_calibration = CalibrationForAnipose3DTracking(single_cams_to_calibrate = single_cams)

In [7]:
#anipose_calibration.load_calibration(filepath = test_data_dir.joinpath('anipose_calibration_all_cams.toml'))

In [15]:
# define function that loops on calibration files
import os
import pandas as pd
import numpy as np
from itertools import cycle 

#calibration_dir = Path('test_data/evaluate_calibration/')

def squared_mean(x):    
    return np.mean(np.array(x)**2)

def get_calibration(anipose_calibration):   
    calibration_file = './test_data/evaluate_calibration/calibration_01/anipose_calibration_all_cams.toml'
    anipose_calibration.load_calibration(filepath = calibration_file)        
    return anipose_calibration, calibration_file

def recalibrate(anipose_calibration, calibration_id, calibration_dir):    
    anipose_calibration.run_calibration()
    filename = 'calibration_{}.toml'.format(calibration_id)
    anipose_calibration.save_calibration(calibration_dir + filename)
    return anipose_calibration, filename

def find_optimal_calibration(anipose_calibration, 
                             test_positions, 
                             calibration_dir,
                             p_threshold=0.1,
                             max_iters=10,
                             error_func=np.mean
                            ):
    """ Finds optimal calibration in folder of calibration files based on mean distance error (mde) """

    # empty dataframe to store calibrations
    calibration_distances = pd.DataFrame(columns=None, index=None)
    calibration_percentages = pd.DataFrame(columns=None, index=None)

    # defining a generator for calibration files (!!not needed if running new calibration on each loop!!)
    calibration_files = cycle(os.listdir(calibration_dir))
    
    for i in range(max_iters):
    
        # get next calibration - !! this needs to be updated with running a new calibration !!        
        anipose_calibration, calibration = get_calibration(anipose_calibration)       
        #anipose_calibration, calibration = recalibrate(anipose_calibration, i, calibration_dir)
            
        # loads calibration and computes distance errors
        anipose_calibration.evaluate_triangulation_of_test_position_markers(test_positions_gt = test_positions, show_3D_plot=False, verbose=False)                
        calibration_errors = anipose_calibration.anipose_io['distance_errors_in_cm']        

        # stores all mean errors for given reference point
        for reference in calibration_errors.keys():

            # extracting distances and percentages
            all_distance_errors = [distance_error for marker_id_a, marker_id_b, distance_error, percentage_error in calibration_errors[reference]['individual_errors']]
            all_percentage_errors = [percentage_error for marker_id_a, marker_id_b, distance_error, percentage_error in calibration_errors[reference]['individual_errors']]

            # store all the distances and percentages
            calibration_distances.loc[calibration,reference] = error_func(all_distance_errors) #calibration_errors[reference]['mean_error']
            calibration_percentages.loc[calibration,reference] = error_func(all_percentage_errors) #calibration_errors[reference]['mean_error']

        print(calibration_percentages.loc[calibration].mean())
        
        # break the loop if percentage error is smaller than threshold
        if calibration_percentages.loc[calibration].mean() < p_threshold:
            break
        
    # average distance errors
    calibration_distances['average'] = calibration_distances.mean(axis=1)
    calibration_distances = calibration_distances.sort_values('average')
    
    # average percentage errors
    calibration_percentages['average'] = calibration_percentages.mean(axis=1)
    calibration_percentages = calibration_percentages.sort_values('average')    

    # finding optimal as min mean across reference points
    optimal_calibration = calibration_percentages['average'].idxmin()    
    #anipose_calibration.load_calibration(filepath = Path(calibration_dir).joinpath(optimal_calibration))
    
    # return the anipose object with optimal calibration implemented
    return optimal_calibration, calibration_distances, calibration_percentages
   

calibration_dir = './test_data/generated_calibrations/'
#calibration_dir = test_data_dir
opt_cal, scores, percentages = find_optimal_calibration(anipose_calibration, 
                                                                             config, 
                                                                             './test_data/generated_calibrations/',
                                                                             p_threshold=0.05,
                                                                             max_iters=2,)



100%|███████████████████████████████| 15/15 [00:00<00:00, 6000.43it/s]


0.11415153118414345


100%|███████████████████████████████| 15/15 [00:00<00:00, 5506.26it/s]


0.11415153118414345


In [16]:
percentages

,screw1_bottom_screw2_bottom,screw1_bottom_screw1_top,screw1_bottom_screw1_nut,screw1_bottom_screw3_bottom,screw1_bottom_screw4_bottom,screw2_bottom_screw3_bottom,screw2_bottom_screw4_bottom,screw1_top_screw1_bottom,screw1_top_screw1_nut,screw1_top_screw2_top,...,maze_corner_closed_left_maze_corner_open_left,maze_corner_closed_left_maze_corner_closed_right,maze_corner_closed_left_maze_corner_open_right,maze_corner_open_right_maze_corner_closed_right,maze_corner_open_right_maze_corner_open_left,maze_corner_open_right_maze_corner_closed_left,maze_corner_closed_right_maze_corner_closed_left,maze_corner_closed_right_maze_corner_open_right,maze_corner_closed_right_maze_corner_open_left,average
./test_data/evaluate_calibration/calibration_01/anipose_calibration_all_cams.toml,0.079089,0.076215,0.112831,0.074672,0.07919,0.096485,0.112167,0.076215,0.129799,0.083166,...,0.07966,0.408727,0.084527,0.082857,0.181,0.084527,0.408727,0.082857,0.080048,0.114152


In [17]:
scores

,screw1_bottom_screw2_bottom,screw1_bottom_screw1_top,screw1_bottom_screw1_nut,screw1_bottom_screw3_bottom,screw1_bottom_screw4_bottom,screw2_bottom_screw3_bottom,screw2_bottom_screw4_bottom,screw1_top_screw1_bottom,screw1_top_screw1_nut,screw1_top_screw2_top,...,maze_corner_closed_left_maze_corner_open_left,maze_corner_closed_left_maze_corner_closed_right,maze_corner_closed_left_maze_corner_open_right,maze_corner_open_right_maze_corner_closed_right,maze_corner_open_right_maze_corner_open_left,maze_corner_open_right_maze_corner_closed_left,maze_corner_closed_right_maze_corner_closed_left,maze_corner_closed_right_maze_corner_open_right,maze_corner_closed_right_maze_corner_open_left,average
./test_data/evaluate_calibration/calibration_01/anipose_calibration_all_cams.toml,0.521121,0.620141,1.01393,0.776375,1.137812,1.597504,1.930958,0.620141,2.24735,0.532403,...,0.511609,5.634757,0.548906,0.528461,2.133963,0.548906,5.634757,0.528461,0.510539,1.386108
